In [1]:
import pickle

from src.evaluation.evaluator_manager import EvaluatorManager
from src.evaluation.evaluator_manager_do import EvaluatorManager as PairedEvaluatorManager
from src.evaluation.evaluator_manager_triplets import EvaluatorManager as TripletsEvaluatorManager

from src.utils.context import Context

from utils_martina.my_utils import *

In [2]:
# Set patient and record (same channels)

# observations = ["chb01_03", "chb01_04", "chb01_15", "chb02_16", "chb02_19", "chb03_01", "chb03_02", "chb03_03", "chb04_28", "chb04_08", "chb06_01", "chb06_04"]

# observations = ["chb01_03", "chb01_04", "chb01_15", "chb01_16", "chb01_18", "chb01_21", "chb01_26"]
# observations = ["chb02_16", "chb02_19"]
# observations = ["chb03_01", "chb03_02", "chb03_03", "chb03_04", "chb03_34", "chb03_35", "chb03_36"]
# observations = ["chb04_28", "chb04_08"]
# observations = ["chb05_06", "chb05_13", "chb05_16", "chb05_17", "chb05_22"]
# observations = ["chb06_01", "chb06_04"] #, "chb06_09", "chb06_10", "chb06_13", "chb06_18", "chb06_24"]

# observations = ["PN00_1", "PN00_2", "PN00_4", "PN00_5"]
# observations = ["PN14_1", "PN14_2", "PN14_3", "PN14_4"]
observations = ["PN14_4"]
# observations = ["chb04_28"]

create_dataset_json(observations)

In [3]:
logs_path = "..\\..\\explainability\GRETEL-repo\\output\\logs\\"
eval_manager_path = "..\\..\\explainability\GRETEL-repo\\output\\eval_manager\\"

## Removing cache dataset

In [4]:
remove_cache_dataset = True

if remove_cache_dataset:
    folder = ".\data\cache\datasets"

    for file in os.listdir(folder):
        if file.startswith("GCS-"):  # Checks if the file begins with "GCS-"
            cache_path = os.path.join(folder, file)
            if os.path.isfile(cache_path):
                os.remove(cache_path)
                print(f"Removed: {cache_path}")

Removed: .\data\cache\datasets\GCS-28e49373646e3a1aa4f78d3b3cf52c7d


## Run Gretel experiment

In [5]:
# NOTICE: If it crashes, check if there are any OracleTorch files from the future!

In [6]:
path = 'config/GCS-GCN.jsonc'
# path = 'config/GCS-SVM.jsonc'
# path = 'config/GCS-KNN.jsonc'

print(observations)

print(f"Generating context for: {path}")
context = Context.get_context(path)
context.run_number = -1

context.logger.info(f"Executing: {context.config_file} Run: {context.run_number}")
context.logger.info(
    "Creating the evaluation manager......................................................."
)

if 'doe-triplets' in context.conf:
    context.logger.info("Creating the TRIPLET evaluators........................................................")
    eval_manager = TripletsEvaluatorManager(context)
if 'do-pairs' in context.conf:
    context.logger.info("Creating the PAIRED evaluators...............................................................") ### Usiamo questo
    eval_manager = PairedEvaluatorManager(context)
else:
    context.logger.info("Creating the evaluators...............................................................")
    eval_manager = EvaluatorManager(context)

context.logger.info(
    "Evaluating the explainers............................................................."
)

eval_manager.evaluate()

# Additional stuff
# context.logger.info(f"graph_ids: {eval_manager._evaluators[0].get_instance_and_counterfactual_ids()}") # NON È STATO SISTEMATO PER CONTEMPLARE IL CASO DI PIU' PAZIENTI
context.logger.info(f"graph_metrics: {eval_manager._evaluators[0].get_instance_and_counterfactual_graph_metrics()}")

['PN14_4']
Generating context for: config/GCS-GCN.jsonc
2025-03-28 18:02:02,814 | INFO | 2000 - Executing: config/GCS-GCN.jsonc Run: -1
2025-03-28 18:02:02,887 | INFO | 2000 - Creating the evaluation manager.......................................................
2025-03-28 18:02:02,981 | INFO | 2000 - Creating the PAIRED evaluators...............................................................
2025-03-28 18:02:03,197 | INFO | 2000 - Creating: GCS-28e49373646e3a1aa4f78d3b3cf52c7d
2025-03-28 18:02:03,200 | INFO | 2000 - Instantiating: src.dataset.generators.gcs.GCS
2025-03-28 18:02:11,270 | INFO | 2000 - Saved: GCS-28e49373646e3a1aa4f78d3b3cf52c7d
2025-03-28 18:02:11,326 | INFO | 2000 - Created: GCS-28e49373646e3a1aa4f78d3b3cf52c7d
2025-03-28 18:02:11,342 | INFO | 2000 - Instantiating: src.oracle.nn.gcn.DownstreamGCN
2025-03-28 18:02:11,358 | INFO | 2000 - Instantiating: torch.optim.RMSprop
2025-03-28 18:02:11,362 | INFO | 2000 - Instantiating: torch.nn.CrossEntropyLoss
2025-03-28 18:02:

In [7]:
# Save eval_manager to file
file_name = get_most_recent_file(logs_path).split('.')[0]

with open(eval_manager_path + file_name + ".pkl", 'wb') as f:
    pickle.dump(eval_manager, f)

## _______________________________________________________________

In [8]:
import datetime

now = datetime.datetime.now()
print("Last full run:", now.strftime("%d/%m/%Y, ore %H:%M"))

Last full run: 28/03/2025, ore 18:13


In [9]:
import winsound

winsound.Beep(440, 500)
winsound.Beep(400, 500)
winsound.Beep(440, 500)

In [10]:
# from src.data_analysis.future.data_analyzer import DataAnalyzer as data_analyzer

# results_path = "output\\results"
# stats_file_path = "output"
# res = data_analyzer.create_aggregated_dataframe(results_path)
# res.to_csv(stats_file_path)
# res